In [1]:
!pip install langchain neo4j openai wikipedia tiktoken langchain_openai pdfplumber

In [2]:
#Package Imports
from langchain.graphs import Neo4jGraph
import neo4j

In [3]:
#Neo4j connection
url = "neo4j+s://39edf771.databases.neo4j.io"
username ="neo4j"
password = "31Nwe5MwJKLGHFCTtkmWQVO7R3DU1fYYvX_D63HZGEM"
graph = Neo4jGraph(
    url=url,
    username=username,
    password=password
)

In [4]:
#Langchain for graph
from langchain_community.graphs.graph_document import (
    Node as BaseNode,
    Relationship as BaseRelationship,
    GraphDocument,
)
from langchain.schema import Document
from typing import List, Dict, Any, Optional
from langchain.pydantic_v1 import Field, BaseModel

class Property(BaseModel):
  """A single property consisting of key and value"""
  key: str = Field(..., description="key")
  value: str = Field(..., description="value")

class Node(BaseNode):
    properties: Optional[List[Property]] = Field(
        None, description="List of node properties")

class Relationship(BaseRelationship):
    properties: Optional[List[Property]] = Field(
        None, description="List of relationship properties"
    )

class KnowledgeGraph(BaseModel):
    """Generate a knowledge graph with entities and relationships."""
    nodes: List[Node] = Field(
        ..., description="List of nodes in the knowledge graph")
    rels: List[Relationship] = Field(
        ..., description="List of relationships in the knowledge graph"
    )

In [5]:
def format_property_key(s: str) -> str:
    words = s.split()
    if not words:
        return s
    first_word = words[0].lower()
    capitalized_words = [word.capitalize() for word in words[1:]]
    return "".join([first_word] + capitalized_words)

def props_to_dict(props) -> dict:
    """Convert properties to a dictionary."""
    properties = {}
    if not props:
      return properties
    for p in props:
        properties[format_property_key(p.key)] = p.value
    return properties

def map_to_base_node(node: Node) -> BaseNode:
    """Map the KnowledgeGraph Node to the base Node."""
    properties = props_to_dict(node.properties) if node.properties else {}
    # Add name property for better Cypher statement generation
    properties["name"] = node.id.title()
    return BaseNode(
        id=node.id.title(), type=node.type.capitalize(), properties=properties
    )


def map_to_base_relationship(rel: Relationship) -> BaseRelationship:
    """Map the KnowledgeGraph Relationship to the base Relationship."""
    source = map_to_base_node(rel.source)
    target = map_to_base_node(rel.target)
    properties = props_to_dict(rel.properties) if rel.properties else {}
    return BaseRelationship(
        source=source, target=target, type=rel.type, properties=properties
    )

In [6]:
import openai
import json
import os
from langchain.chat_models import AzureChatOpenAI
from langchain.schema import HumanMessage
from langchain import LLMChain
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Load config values
with open(r'config.json') as config_file:
    config_details = json.load(config_file)

# The base URL for your Azure OpenAI resource.
openai_api_base = config_details['OPENAI_API_BASE']

# API version e.g. "2023-07-01-preview"
openai_api_version = config_details['OPENAI_API_VERSION']

# The name of your Azure OpenAI deployment chat model. e.g. "gpt-35-turbo-0613"
deployment_name = config_details['DEPLOYMENT_NAME']

# The API key for your Azure OpenAI resource.
openai_api_key = os.getenv("OPENAI_API_KEY")

# This is set to `azure`
openai_api_type = "azure"

In [7]:
from langchain.chat_models import AzureChatOpenAI

# Create an instance of chat llm
llm = AzureChatOpenAI(
    azure_endpoint=openai_api_base,
    openai_api_version=openai_api_version,
    deployment_name=deployment_name,
    openai_api_key=openai_api_key,
    openai_api_type=openai_api_type,
)

c:\Users\WendyChuaXingZhao\anaconda3\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureChatOpenAI`.
  warn_deprecated(


In [8]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.openai_functions import create_structured_output_chain

def get_extraction_chain(
    allowed_nodes: Optional[List[str]] = None,
    allowed_rels: Optional[List[str]] = None
    ):
    prompt = ChatPromptTemplate.from_messages(
    [(
      "system",
      f"""# Knowledge Graph Instructions for GPT-4
## 1. Overview
You are a specialized algorithm designed to extract structured financial data from bank annual reports to build a comprehensive knowledge graph.
- **Nodes** represent financial terms, entities, departments, and concepts specific to the banking industry.
- The goal is to structure information in a manner that highlights financial relationships, decision-making hierarchies, and policy summaries.

## 2. Sections and Nodes
- **Sections**: Target key sections such as 'Director's Report', 'Statements of Financial Position', 'Income Statement', etc. Identify relevant financial data and decisions.
- **Entities and Concepts**: Create nodes for identifiable entities like board members, financial metrics, and policy terms.
- **Node IDs**: Use names or specific identifiers for node labels. Avoid integers or vague references.
- **Relationships**: Map relationships that reflect financial dependencies, reporting structures, and policy impacts.

## 3. Detailed Financial Extraction
- **Extract Details**: Focus on numbers and financial statements, converting them into properties of nodes. For instance, assets and liabilities figures should be attached to the 'Statement of Financial Position' node.
- **Accuracy and Precision**: Ensure the extraction of financial data is accurate, citing exact figures and contextual information.
- **Contextual Relevance**: Attach relevant notes and assumptions from the 'Notes to Financial Statements' to the appropriate financial statements or metrics.

## 4. Compliance and Consistency
- **Regulatory Statements**: Extract and highlight compliance statements from 'Independent Auditors' Report' and 'Statutory Declaration'.
- **Maintain Consistency**: Use consistent terminology across different sections of the report to avoid confusion.
- **Strict Compliance**: Adhere strictly to the rules for knowledge graph construction.

## 5. Symbolic Tokens and Special Characters
- **Symbolic Tokens**: Identify and properly handle symbolic tokens (e.g., %, $, £, etc.) and ensure they are accurately represented in the data.
- **Special Characters**: Recognize special characters and their meanings (e.g., ± for approximately, > for greater than, etc.) and handle them appropriately in the extraction process.

## 6. Domain Vocabulary and Abbreviations
- **Domain Vocabulary**: Utilize and recognize domain-specific vocabulary and terminology relevant to the banking and financial industry.
- **Abbreviations**: Identify and expand domain-related abbreviations (e.g., FVOCI for Fair Value through Other Comprehensive Income) to ensure clarity and accuracy.

## 7. N-gram Extraction
- **N-grams**: Extract meaningful N-grams (bigrams, trigrams, etc.) that represent significant financial terms or phrases and include them as part of the node properties.

## 8. Tips
- Remember to format financial data as attributes of the nodes and structure the graph to reflect the organization and flow of the annual report.
- Use the given format to extract information from the following input: <input here>
- Tip: Make sure to answer in the correct format
"""),
        ("human", "Use the given format to extract information from the following input: {input}"),
        ("human", "Tip: Make sure to answer in the correct format"),
    ])
    return create_structured_output_chain(KnowledgeGraph, llm, prompt, verbose=False)


In [9]:
def extract_and_store_graph(
    document: Document,
    nodes: Optional[List[str]] = None,
    rels: Optional[List[str]] = None
) -> None:
    # Extract graph data using OpenAI functions
    extract_chain = get_extraction_chain(nodes, rels)
    data = extract_chain.invoke(document)['function']
    
    # Construct a graph document
    graph_document = GraphDocument(
        nodes=[map_to_base_node(node) for node in data.nodes],
        relationships=[map_to_base_relationship(rel) for rel in data.rels],
        source=document.dict()  # Convert Document object to dictionary
    )
    
    # Store information into a graph
    graph.add_graph_documents([graph_document])


In [10]:
import pdfplumber
import os
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from langchain.text_splitter import TokenTextSplitter
from tqdm import tqdm
import pickle

# Ensure nltk resources are downloaded
nltk.download('stopwords')
nltk.download('wordnet')

# Define the folder containing the PDF files
pdf_folder_path = r"C:\Users\WendyChuaXingZhao\OneDrive - SRKK Group of Companies\Documents\MsDS\Knowledge Graph Test\Financial Documents"

def extract_text_and_tables_from_pdf(pdf_path):
    try:
        with pdfplumber.open(pdf_path) as pdf:
            full_text = ''
            tables = []
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text:
                    full_text += page_text
                page_tables = page.extract_tables()
                if page_tables:
                    tables.extend(page_tables)
        return full_text, tables
    except Exception as e:
        print(f"An error occurred while reading the PDF {pdf_path}: {e}")
        return None, None

def preprocess_text(text):
    # Convert text to lower case
    text = text.lower()
    # Special strings removal
    text = re.sub(r'[^a-z0-9\s]+', ' ', text)
    
    # Tokenization
    tokens = text.split()
    
    # Stop-word removal
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word.lower() not in stop_words]
    
    # Stemming and Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    # Join tokens back to string
    preprocessed_text = ' '.join(tokens)
    return preprocessed_text

def extract_and_preprocess_pdf(pdf_path):
    raw_text, _ = extract_text_and_tables_from_pdf(pdf_path)
    if raw_text:
        preprocessed_text = preprocess_text(raw_text)
        text_splitter = TokenTextSplitter(chunk_size=2048, chunk_overlap=24)
        documents = text_splitter.split_text(preprocessed_text)
        return documents
    return []

# List all PDF files in the directory
pdf_paths = [os.path.join(pdf_folder_path, f) for f in os.listdir(pdf_folder_path) if f.lower().endswith('.pdf')]

# Extract and preprocess documents
all_documents = []
for pdf_path in pdf_paths:
    documents = extract_and_preprocess_pdf(pdf_path)
    all_documents.extend(documents)

# Save preprocessed documents for later use
with open('processed_documents.pkl', 'wb') as f:
    pickle.dump(all_documents, f)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\WendyChuaXingZhao\AppData\Roaming\nltk_data..
[nltk_data]     .
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\WendyChuaXingZhao\AppData\Roaming\nltk_data..
[nltk_data]     .
[nltk_data]   Package wordnet is already up-to-date!


In [11]:
import pickle
from tqdm import tqdm
from langchain.schema import Document
import json
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, filename='error_log.log', filemode='w',
                    format='%(name)s - %(levelname)s - %(message)s')

# Load preprocessed documents
with open('processed_documents.pkl', 'rb') as f:
    documents = pickle.load(f)

def extract_and_store_graph(
    document: Document,
    nodes: Optional[List[str]] = None,
    rels: Optional[List[str]] = None
) -> None:
    try:
        # Extract graph data using OpenAI functions
        extract_chain = get_extraction_chain(nodes, rels)
        response = extract_chain.invoke(document)
        
        # Log the response for debugging
        logging.info("Response from extract_chain: %s", response)
        
        # Validate JSON response
        try:
            data = response['function']
            # Construct a graph document
            graph_document = GraphDocument(
                nodes=[map_to_base_node(node) for node in data.nodes],
                relationships=[map_to_base_relationship(rel) for rel in data.rels],
                source=document.dict()  # Convert Document object to dictionary
            )
            
            # Store information into a graph
            graph.add_graph_documents([graph_document])
        
        except json.JSONDecodeError as e:
            logging.error("JSON decoding error: %s", e)
            logging.error("Invalid JSON response: %s", response)
        except KeyError as e:
            logging.error("Key error: %s", e)
            logging.error("Response missing expected key: %s", response)
        except Exception as e:
            logging.error("An unexpected error occurred: %s", e)
            logging.error("Response: %s", response)
    
    except Exception as e:
        logging.error("An error occurred in extract_and_store_graph: %s", e)

# Process each document chunk and extract/store the graph
for i, chunk in tqdm(enumerate(documents), total=len(documents)):
    doc = Document(page_content=chunk, metadata={"source": "batch_process", "chunk": i})
    try:
        extract_and_store_graph(doc)
        print(f"Successfully processed and stored chunk {i+1}")
    except Exception as e:
        print(f"An error occurred while processing chunk {i+1}: {e}")


  0%|          | 0/3412 [00:00<?, ?it/s]c:\Users\WendyChuaXingZhao\anaconda3\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The function `create_structured_output_chain` was deprecated in LangChain 0.1.1 and will be removed in 0.3.0. Use ChatOpenAI.with_structured_output instead.
  warn_deprecated(
  0%|          | 1/3412 [00:10<10:15:37, 10.83s/it]

Successfully processed and stored chunk 1


  0%|          | 2/3412 [00:42<22:05:53, 23.33s/it]

Successfully processed and stored chunk 2


  0%|          | 3/3412 [01:06<22:13:52, 23.48s/it]

...

Successfully processed and stored chunk 556


 16%|█▋        | 557/3412 [9:26:50<59:19:18, 74.80s/it]

Successfully processed and stored chunk 557


 16%|█▋        | 558/3412 [9:26:59<43:34:14, 54.96s/it]

Successfully processed and stored chunk 558
